In [18]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
tf.random.set_seed(42)

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 4

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [5]:
class_names = ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'no_tumor']

In [20]:
def create_data_generator(data_dir, generator, subset):
    return generator.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=class_names,
        shuffle=True if subset == 'training' else False,
        subset=subset
    )

In [10]:
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [11]:
data_dir = 'Training'

In [21]:
train_generator = create_data_generator(data_dir, train_datagen, 'training')
validation_generator = create_data_generator(data_dir, val_datagen, 'validation')

Found 2297 images belonging to 4 classes.
Found 573 images belonging to 4 classes.


In [23]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


In [24]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

In [25]:
model = Model(inputs=base_model.input, outputs=output)


In [26]:
for layer in base_model.layers:
    layer.trainable = False

In [27]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [29]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/20


C:\Users\kusha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


71/71 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.5323 - loss: 1.0562 - val_accuracy: 0.7059 - val_loss: 0.7871
Epoch 2/20
 1/71 ━━━━━━━━━━━━━━━━━━━━ 3:14 3s/step - accuracy: 0.8125 - loss: 0.5255

C:\Users\kusha\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


71/71 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.8125 - loss: 0.5255 - val_accuracy: 0.6897 - val_loss: 0.7375
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 222s 3s/step - accuracy: 0.7606 - loss: 0.6257 - val_accuracy: 0.7500 - val_loss: 0.6614
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.7188 - loss: 0.6219 - val_accuracy: 0.5517 - val_loss: 0.9820
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 411s 6s/step - accuracy: 0.7806 - loss: 0.5621 - val_accuracy: 0.7647 - val_loss: 0.6714
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.8750 - loss: 0.5212 - val_accuracy: 0.5862 - val_loss: 1.0049
Epoch 7/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 227s 3s/step - accuracy: 0.7803 - loss: 0.5676 - val_accuracy: 0.7868 - val_loss: 0.6530
Epoch 8/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.8125 - loss: 0.5263 - val_accuracy: 0.6552 - val_loss: 0.8438
Epoch 9/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 0.7994 - loss: 0.5296 - val_accuracy: 0.7941 - val_loss: 0.

In [31]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.7183 - loss: 0.8385
Test accuracy: 0.7714
Test loss: 0.7077


In [33]:
model.save('brain_mri_inceptionv2_model.keras')
